**최근에 데스크탑, 노트북 구매에 관하여 관심이 생긴 김에 관련 주제로 프로젝트를 해보자 라는 생각을 하였습니다.**

In [1]:
pip install category_encoders

In [2]:
! pip install pandas-profiling

In [3]:
import pandas as pd
import numpy as np
from pandas_profiling import ProfileReport

df = pd.read_csv('LaptopSales.csv')
df_original = pd.read_csv('LaptopSales.csv')

df.head()

,Date,Configuration,Customer Postcode,Store Postcode,Retail Price,Screen Size (Inches),Battery Life (Hours),RAM (GB),Processor Speeds (GHz),Integrated Wireless?,HD Size (GB),Bundled Applications?,customer X,customer Y,store X,store Y
0,01/01/2008 5:47,1,E1 6NU,E2 0RY,370.0,15,4,1,1.5,Yes,40,Yes,533579,182085,535652.0,182961.0
1,01/02/2008 1:59,1,SE26 5QF,SE8 3JD,370.0,15,4,1,1.5,Yes,40,Yes,535546,171459,537175.0,177885.0
2,01/02/2008 21:35,1,SW15 1JN,SW18 1NN,370.0,15,4,1,1.5,Yes,40,Yes,523931,175760,525155.0,175180.0
3,01/03/2008 19:44,1,W1U 2SA,NW5 2QH,370.0,15,4,1,1.5,Yes,40,Yes,528476,181338,529248.0,185213.0
4,01/04/2008 15:02,1,SW3 2DY,SW1P 3AU,370.0,15,4,1,1.5,Yes,40,Yes,527238,178835,529902.0,179641.0


In [4]:
df = df.drop(['Date', 'Configuration', 'Customer Postcode', 'Store Postcode', 'customer X', 'customer Y', 'store X', 'store Y'], axis = 1)
df = df.dropna(axis=0)
df.head()

,Retail Price,Screen Size (Inches),Battery Life (Hours),RAM (GB),Processor Speeds (GHz),Integrated Wireless?,HD Size (GB),Bundled Applications?
0,370.0,15,4,1,1.5,Yes,40,Yes
1,370.0,15,4,1,1.5,Yes,40,Yes
2,370.0,15,4,1,1.5,Yes,40,Yes
3,370.0,15,4,1,1.5,Yes,40,Yes
4,370.0,15,4,1,1.5,Yes,40,Yes


In [5]:
from category_encoders import OneHotEncoder
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error

#train,test 데이터 나누기
train = df.sample(frac=0.75,random_state=1)
test = df.drop(train.index)

# Retail Price 평균
train['Retail Price'].mean()

target = 'Retail Price'
y_train = train[target]
y_test = test[target]

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


**전 이 데이터에서의 타겟값을 'Retail Price' 로 잡았습니다.이유는 이 데이터를 분석하는 최종 목적이 가격이라고 생각하기 때문입니다 또한 사용 할 ML모델은 선형회귀모델 중에서 다중선형회귀모델을 사용하기로 결정하였습니다.이 결정의 이유는 타겟값인 노트북의 가격이라는 수치이가 여러 요소에 영향을 받을 것이라고 생각했기 때문입니다.**

**또한 기준모델을 평균값으로 정한 이유는 제가 사용할 모델이 회귀모델이기 때문입니다.**

In [6]:
# Retail Price 평균값으로 기준모델 설정 
predict = y_train.mean()
y_pred = [predict] * len(y_train)
mae = mean_absolute_error(y_train, y_pred)

print(f'훈련 에러: {mae:.2f}')

훈련 에러: 82.13


In [7]:
y_pred = [predict] * len(y_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'테스트 에러: {mae:.2f}')

테스트 에러: 81.51


In [8]:
#다중선형회귀모델 학습을 위한 특성
features = ['Processor Speeds (GHz)','RAM (GB)', 'Screen Size (Inches)']

X_train = train[features]
X_test = test[features]

In [9]:
model = LinearRegression()

In [10]:
#모델에 fit
model.fit(X_train, y_train)
y_pred = model.predict(X_train)
mae = mean_absolute_error(y_train, y_pred)
print(f'훈련 에러: {mae:.2f}')

훈련 에러: 61.03


In [11]:
#테스트 데이터에 대하여 적용
y_pred = model.predict(X_test)
mae = mean_absolute_error(y_test, y_pred)
print(f'테스트 에러: {mae:.2f}')

테스트 에러: 61.05


**기준모델의 에러는 훈련에러 82.13 테스트에러 81.51  다중선형회귀모델에서의 에러는 훈련에러 61.03, 테스트에러 61.05 로 기준모델보다 확연히 줄어든 것을 볼 수 있습니다.**

In [12]:
#모델의 회귀계수와 절편,그리고 회귀 식
model.intercept_, model.coef_

(-384.05628233451233, array([53.52081243, 47.03570768, 43.63398168]))

In [13]:
b0 = model.intercept_
b1, b2, b3 = model.coef_

print(f'y = {b0:.0f} + {b1:.0f}x\u2081 + {b2:.0f}x\u2082 + {b3:.0f}x\u2083')

y = -384 + 54x₁ + 47x₂ + 44x₃


**여기의 회귀 식을 보면 Y값 즉 노트북의 가격은 처음에 설정한 요소 3개에 형향을 받는 다는 것을 알 수 있습니다.**

In [14]:
#평가지표 계산
from sklearn.metrics import  mean_squared_error, mean_absolute_error, r2_score
from IPython.display import display
from matplotlib.patches import Rectangle
import matplotlib.pyplot as plt

def evaluation_metrics(df, feature, target, slope, intercept):

    
    x = df[feature]
    y = df[target]
    
    y_pred = slope * x + intercept
    
    
    
    mse = mean_squared_error(y, y_pred)
    mae = mean_absolute_error(y, y_pred)
    rmse = mse ** 0.5
    r2 = r2_score(y, y_pred)

    display(pd.DataFrame([['MSE', mse],['MAE', mae],['RMSE', rmse],['R2', r2]], columns=['Metric', 'Score']))

In [15]:
evaluation_metrics(df, 'Processor Speeds (GHz)', 'Retail Price', 0, df['Retail Price'].mean())

,Metric,Score
0,MSE,10943.587891
1,MAE,81.986978
2,RMSE,104.611605
3,R2,0.000000


In [16]:
evaluation_metrics(df, 'Processor Speeds (GHz)', 'Retail Price', 750,0 )   

,Metric,Score
0,MSE,956487.375217
1,MAE,938.840671
2,RMSE,978.001726
3,R2,-86.401626


In [17]:
evaluation_metrics(df, 'RAM (GB)', 'Retail Price', 100,10 ) 

,Metric,Score
0,MSE,95160.622034
1,MAE,288.494522
2,RMSE,308.481153
3,R2,-7.695560


In [18]:
evaluation_metrics(df, 'Screen Size (Inches)', 'Retail Price', 2,10 ) 

,Metric,Score
0,MSE,228416.039890
1,MAE,466.505855
2,RMSE,477.928907
3,R2,-19.872135


In [98]:
!pip install pdpbox

     |████████████████████████████████| 34.0MB 80kB/s 
     |████████████████████████████████| 13.1MB 226kB/s 
  Created wheel for pdpbox: filename=PDPbox-0.2.1-cp37-none-any.whl size=35758227 sha256=33b3ca629ea2b42319500a9556d86487baa391e089a4f7bb9806f4a224887611
  Stored in directory: /root/.cache/pip/wheels/c8/13/11/ecac74c6192790c94f6f5d919f1a388f927b9febb690e45eef
Successfully built pdpbox
ERROR: albumentations 0.1.12 has requirement imgaug<0.2.7,>=0.2.5, but you'll have imgaug 0.2.9 which is incompatible.
  Found existing installation: matplotlib 3.2.2
    Uninstalling matplotlib-3.2.2:
      Successfully uninstalled matplotlib-3.2.2


In [24]:
from pdpbox.pdp import pdp_isolate, pdp_plot

feature = 'Processor Speeds (GHz)'

isolated = pdp_isolate(
    model, 
    feature=feature,
)
pdp_plot(isolated, feature_name=feature);

TypeError: ignored